In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import boto3
from botocore.client import Config
import os
import pandas as pd
from io import StringIO
import matplotlib.dates as dates
from datetime import datetime
import matplotlib.ticker as ticker
from random import shuffle
from time import time
from pandas.plotting import register_matplotlib_converters

os.chdir('C:\\Users\\A Sua\\Documents\\AWS\\credentials')
list_of_lines = []
with open('capstoner_key.txt') as fp:
    lines = fp.readlines()
    for line in lines:
        line = line.strip()
        list_of_lines.append(line)
    fp.close()


location = list_of_lines[0].split('=')[1]
AWS_ACCESS_KEY_ID = list_of_lines[1].split('=')[1]
AWS_SECRET_ACCESS_KEY = list_of_lines[2].split('=')[1]
os.chdir('C:\\Users\\A Sua\\Documents\\FIU\\CAP2020')
%matplotlib notebook

In [3]:
register_matplotlib_converters()
%matplotlib notebook

In [4]:
df_name_list = ['df_returns', 'df_price', 'df_spread', 'df_volume', 'df_shares_out']

In [5]:
### import returns
### import price
### import spread
### import volume
### import sharesout
### make spread/price
### make volume/sharesout

In [6]:
df_list = []
for df_name in df_name_list:
    ### BUCKETS & KEYS
    BUCKET_NAME = 'capstonefiu2020-data'
    key_location = 'data/processed'
    file_name = '{}07_18.csv'.format(df_name)
    key_path = key_location + '/' + file_name

    ### ACTIVATE CLIENT
    client = boto3.client('s3', 
                  aws_access_key_id = AWS_ACCESS_KEY_ID,
                   aws_secret_access_key=AWS_SECRET_ACCESS_KEY)


    ### INSTANTIATE DATAFRAME
    csv_obj = client.get_object(Bucket = BUCKET_NAME, Key=key_path)
    body = csv_obj['Body']
    csv_string = body.read().decode('utf-8')

    df = pd.read_csv(StringIO(csv_string))
    df.index = df['Unnamed: 0']
    df = df.drop('Unnamed: 0', axis=1)
    df.index.names = ['date']


    #df_bidlo.head()
    #### TRANSPOSE DATA FOR EASE OF USE
    df = df.T
    df.index = pd.to_datetime(df.index)
    df_list.append(df)
    print('{} has shape: {}'.format(df_name, df.shape))

df_returns has shape: (3020, 388)
df_price has shape: (3020, 388)
df_spread has shape: (3020, 388)
df_volume has shape: (3020, 388)
df_shares_out has shape: (3020, 388)


In [7]:
df_returns = df_list[0]
df_price = df_list[1]
df_spread = df_list[2]
df_volume = df_list[3]
df_sharesout = df_list[4]

In [8]:
#del df_list

In [9]:
df_spread_price = df_spread/df_price
df_volume_sharesout = df_volume/df_sharesout

In [10]:
companies = df_returns.columns
companies[:5]

Index(['ORCL_ORACLE CORP', 'MSFT_MICROSOFT CORP',
       'TROW_T ROWE PRICE GROUP INC', 'HON_HONEYWELL INTERNATIONAL INC',
       'ADM_ARCHER DANIELS MIDLAND CO'],
      dtype='object', name='date')

In [11]:
### Generate Dates

In [12]:
dates = df_sharesout.index
worst_dates_dow = pd.to_datetime(['2008-10-15', '2008-12-1', '2008-10-9', '2008-9-29', '2011-8-8'])
#worst_dates_sp = 
worst_dates_dow

DatetimeIndex(['2008-10-15', '2008-12-01', '2008-10-09', '2008-09-29',
               '2011-08-08'],
              dtype='datetime64[ns]', freq=None)

### What were the metrics of the companies on their 5 worst Trading Days?

### Metrics of the Company for their 5 worst Return dates

#### returns

In [13]:
worst_days_returns = {}
for symbol in df_returns.columns:
    temp_series = df_returns[symbol].sort_values(ascending=True).iloc[:5]
    worst_days_returns[symbol] = temp_series
print(len(worst_days_returns))
worst_days_returns['HON_HONEYWELL INTERNATIONAL INC']

388


2008-10-15   -0.094081
2008-11-06   -0.088738
2008-09-29   -0.086031
2008-10-09   -0.078932
2011-08-08   -0.076907
Name: HON_HONEYWELL INTERNATIONAL INC, dtype: float64

#### price

In [14]:
worst_days_prices = {}
for symbol in df_price:
    temp_dic = worst_days_returns[symbol]
    temp_series = pd.merge(temp_dic, df_price[symbol], left_index=True, right_index=True, copy=False, suffixes=('_x', ''))[symbol]
    worst_days_prices[symbol] = temp_series
print(len(worst_days_prices))
worst_days_prices['HON_HONEYWELL INTERNATIONAL INC']

388


2008-10-15    29.08
2008-11-06    28.24
2008-09-29    40.37
2008-10-09    31.39
2011-08-08    44.29
Name: HON_HONEYWELL INTERNATIONAL INC, dtype: float64

#### spread

In [15]:
worst_days_spread = {}
for symbol in df_spread:
    temp_dic = worst_days_returns[symbol]
    temp_series = pd.merge(temp_dic, df_spread[symbol], left_index=True, right_index=True, copy=False, suffixes=('_x', ''))[symbol]
    worst_days_spread[symbol] = temp_series
print(len(worst_days_spread))
worst_days_spread['HON_HONEYWELL INTERNATIONAL INC']

388


2008-10-15    3.69
2008-11-06    3.23
2008-09-29    4.39
2008-10-09    5.27
2011-08-08    3.15
Name: HON_HONEYWELL INTERNATIONAL INC, dtype: float64

#### spread/price

In [16]:
worst_days_spread_price = {}
for symbol in df_spread:
    temp_dic = worst_days_returns[symbol]
    temp_series = pd.merge(temp_dic, df_spread_price[symbol], left_index=True, right_index=True, copy=False, suffixes=('_x', ''))[symbol]
    worst_days_spread_price[symbol] = temp_series
print(len(worst_days_spread_price))
worst_days_spread_price['HON_HONEYWELL INTERNATIONAL INC']

388


2008-10-15    0.126891
2008-11-06    0.114377
2008-09-29    0.108744
2008-10-09    0.167888
2011-08-08    0.071122
Name: HON_HONEYWELL INTERNATIONAL INC, dtype: float64

#### volume

In [17]:
worst_days_volume = {}
for symbol in df_volume.columns:
    temp_dic = worst_days_returns[symbol]
    temp_series = pd.merge(temp_dic, df_volume[symbol], left_index=True, right_index=True, copy=False, suffixes=('_x', ''))[symbol]
    worst_days_volume[symbol] = temp_series
print(len(worst_days_volume))
worst_days_volume['HON_HONEYWELL INTERNATIONAL INC']

388


2008-10-15    12289000.0
2008-11-06     8030100.0
2008-09-29    10095800.0
2008-10-09    12508700.0
2011-08-08    13255800.0
Name: HON_HONEYWELL INTERNATIONAL INC, dtype: float64

#### sharesout

In [18]:
worst_days_sharesout = {}
for symbol in df_sharesout.columns:
    temp_dic = worst_days_returns[symbol]
    temp_series = pd.merge(temp_dic, df_sharesout[symbol], left_index=True, right_index=True, copy=False, suffixes=('_x', ''))[symbol]
    worst_days_sharesout[symbol] = temp_series
print(len(worst_days_sharesout))
worst_days_sharesout['HON_HONEYWELL INTERNATIONAL INC']

388


2008-10-15    744436.0
2008-11-06    726287.0
2008-09-29    744436.0
2008-10-09    744436.0
2011-08-08    782425.0
Name: HON_HONEYWELL INTERNATIONAL INC, dtype: float64

#### volume/sharesout

In [19]:
worst_days_volume_sharesout = {}
for symbol in df_volume_sharesout.columns:
    temp_dic = worst_days_returns[symbol]
    temp_series = pd.merge(temp_dic, df_volume_sharesout[symbol], left_index=True, right_index=True, copy=False, suffixes=('_x', ''))[symbol]
    worst_days_volume_sharesout[symbol] = temp_series
print(len(worst_days_volume_sharesout))
worst_days_volume_sharesout['HON_HONEYWELL INTERNATIONAL INC']

388


2008-10-15    16.507799
2008-11-06    11.056373
2008-09-29    13.561676
2008-10-09    16.802922
2011-08-08    16.941943
Name: HON_HONEYWELL INTERNATIONAL INC, dtype: float64

### Metrics for the Company on the 5 worst Market/Systemic Return Dates

#### returns

In [22]:
worst_days_sys_returns = {}
for symbol in df_returns.columns:
    temp_returns = df_returns[symbol]
    temp_series = df_returns[symbol]
    temp_series = pd.Series([temp_returns[worst_dates_dow[i]] for i in range(len(worst_dates_dow))], index=[worst_dates_dow])
    worst_days_sys_returns[symbol] = temp_series

worst_days_sys_returns['HON_HONEYWELL INTERNATIONAL INC']

2008-10-15   -0.094081
2008-12-01   -0.067121
2008-10-09   -0.078932
2008-09-29   -0.086031
2011-08-08   -0.076907
dtype: float64

#### price

In [23]:
worst_days_sys_prices = {}
for symbol in df_price.columns:
    temp_series = pd.Series([df_price[symbol][worst_dates_dow[i]] for i in range(len(worst_dates_dow))], index=[worst_dates_dow])
    worst_days_sys_prices[symbol] = temp_series

worst_days_sys_prices['HON_HONEYWELL INTERNATIONAL INC']

2008-10-15    29.08
2008-12-01    25.99
2008-10-09    31.39
2008-09-29    40.37
2011-08-08    44.29
dtype: float64

#### spread

In [24]:
worst_days_sys_spread = {}
for symbol in df_spread.columns:
    temp_series = pd.Series([df_spread[symbol][worst_dates_dow[i]] for i in range(len(worst_dates_dow))], index=[worst_dates_dow])
    worst_days_sys_spread[symbol] = temp_series

worst_days_sys_spread['HON_HONEYWELL INTERNATIONAL INC']

2008-10-15    3.69
2008-12-01    1.87
2008-10-09    5.27
2008-09-29    4.39
2011-08-08    3.15
dtype: float64

#### spread/price

In [25]:
worst_days_sys_spread_price = {}
for symbol in df_spread_price.columns:
    temp_series = pd.Series([df_spread_price[symbol][worst_dates_dow[i]] for i in range(len(worst_dates_dow))], index=[worst_dates_dow])
    worst_days_sys_spread_price[symbol] = temp_series

worst_days_sys_spread_price['HON_HONEYWELL INTERNATIONAL INC']

2008-10-15    0.126891
2008-12-01    0.071951
2008-10-09    0.167888
2008-09-29    0.108744
2011-08-08    0.071122
dtype: float64

#### volume

In [26]:
worst_days_sys_volume = {}
for symbol in df_volume.columns:
    temp_series = pd.Series([df_volume[symbol][worst_dates_dow[i]] for i in range(len(worst_dates_dow))], index=[worst_dates_dow])
    worst_days_sys_volume[symbol] = temp_series

worst_days_sys_volume['HON_HONEYWELL INTERNATIONAL INC']

2008-10-15    12289000.0
2008-12-01     6045100.0
2008-10-09    12508700.0
2008-09-29    10095800.0
2011-08-08    13255800.0
dtype: float64

#### sharesout

In [29]:
worst_days_sys_sharesout = {}
for symbol in df_sharesout.columns:
    temp_series = pd.Series([df_sharesout[symbol][worst_dates_dow[i]] for i in range(len(worst_dates_dow))], index=[worst_dates_dow])
    worst_days_sys_sharesout[symbol] = temp_series

worst_days_sys_sharesout['HON_HONEYWELL INTERNATIONAL INC']

2008-10-15    744436.0
2008-12-01    726287.0
2008-10-09    744436.0
2008-09-29    744436.0
2011-08-08    782425.0
dtype: float64

#### volume/sharesout

In [30]:
worst_days_sys_volume_sharesout = {}
#error_list = []
for symbol in df_volume_sharesout.columns:
    temp_series = pd.Series([df_volume_sharesout[symbol][worst_dates_dow[i]] for i in range(len(worst_dates_dow))], index=[worst_dates_dow])
    worst_days_sys_volume_sharesout[symbol] = temp_series

worst_days_sys_volume_sharesout['HON_HONEYWELL INTERNATIONAL INC']

2008-10-15    16.507799
2008-12-01     8.323294
2008-10-09    16.802922
2008-09-29    13.561676
2011-08-08    16.941943
dtype: float64

### 7 Histograms
### 7 Timeseries per company
##### (388 subplots * 14 graphs/subplot = 5432 graphs)

In [ ]:
#sup_title_font =15
sub_title_font =8
legend_font =8
axis_font = 6
tick_rotation = 20
black_dot = 2
red_dot = 5

### matrix of subplots
n_cols = 7
n_rows = 2
n_plots = n_cols*n_rows
nloops = int(np.floor(388/n_plots))
temp_matrix = np.array(range(n_plots)).reshape(n_rows,n_cols)

fig, ax = plt.subplots(n_cols,n_rows,figsize=(17,12))



plt.tight_layout()